## GRU

Попытка сделать монофонический выход из сетки. Аналогично LSTM

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np

Сделаем также пользовательский импорт

In [11]:
from decode_patterns import data_conversion

Загружаем датасет

In [12]:
# import dataset
drum, bass = data_conversion.make_lstm_dataset(height=16, limit=30000, patterns_file="decode_patterns/patterns.pairs.tsv", mono=True)


# define shuffling of dataset
def shuffle(A, B, p=0.8):
    # take 80% to training, other to testing
    L = len(A)
    idx = np.arange(L) < p*L
    np.random.shuffle(idx)
    yield A[idx]
    yield B[idx]
    yield A[np.logical_not(idx)]
    yield B[np.logical_not(idx)]
    
    
# we can select here a validation set
drum, bass, drum_validation, bass_validation = shuffle(drum, bass)
    
# and we can shuffle train and test set like this:
# drum_train, bass_train, drum_test, bass_test = shuffle(drum, bass)

In [13]:
bass_validation[10]

array([ 4,  1,  4,  4,  4,  1,  0,  0,  0, 14,  0,  0,  0,  0,  0,  0],
      dtype=int64)

In [14]:
# попробуем определить модель LSTM как конечный автомат
class DrumNBassGRU(nn.Module):
    def __init__(self):
        super(DrumNBassGRU, self).__init__()
        # one input neuron, one output neuron, one layer in LSTM block
        self.input_size = 14
        self.hidden_size = 34
        self.layer_count = 1
        self.lstm = nn.GRU(self.input_size, self.hidden_size, self.layer_count)
        self.embed_layer = nn.Linear(self.hidden_size, 1)
        self.sigm = nn.Sigmoid()
    
    def forward(self, input):
        # пусть в input у нас приходит вектор размерности (64, 32, 14)
        # то есть 64 отсчёта, тридцать два примера (минибатч), 14 значение в каждом (барабанная партия)
        output, _ = self.lstm(input)
        output = self.sigm(self.embed_layer(output))*37
        return output

In [15]:
# часть обучения
dnb_gru = DrumNBassGRU()

criterion = nn.MSELoss()

# оценим также и разнообразие мелодии по её.. дисперсии?)
# def melody_variety(melody):
#     return 1/(1 + (melody.sum(axis=2) > 1).int())
    
# criterion = nn.NLLLoss() # -- этот товарищ требует, чтобы LSTM выдавал классы,
# criterion = nn.CrossEntropyLoss() # и этот тоже
# (числа от 0 до C-1), но как всё-таки его заставить это делать?...
# optimizer = optim.SGD(dnb_lstm.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(dnb_gru.parameters(), lr=0.001)

Найденные баги и их решения:

https://stackoverflow.com/questions/56741087/how-to-fix-runtimeerror-expected-object-of-scalar-type-float-but-got-scalar-typ

https://stackoverflow.com/questions/49206550/pytorch-error-multi-target-not-supported-in-crossentropyloss/49209628

https://stackoverflow.com/questions/56243672/expected-target-size-50-88-got-torch-size50-288-88

In [16]:
epoch_count = 500
batch_size = 128
shuffle_every_epoch = True
    
if shuffle_every_epoch:
    print(f"shuffle_every_epoch is on")
else:
    print(f"shuffle_every_epoch is off")
    # shuffle train and test set:
    drum_train, bass_train, drum_test, bass_test = shuffle(drum, bass)
    drum_train = torch.tensor(drum_train, dtype=torch.float)
    bass_train = torch.tensor(bass_train, dtype=torch.float)
    drum_test = torch.tensor(drum_test, dtype=torch.float)
    drum_test = torch.tensor(drum_test, dtype=torch.float)
        
for epoch in range(epoch_count):  # loop over the dataset multiple times
    print(f"Epoch #{epoch}")
    if shuffle_every_epoch:
        # shuffle train and test set:
        drum_train, bass_train, drum_test, bass_test = shuffle(drum, bass)
        drum_train = torch.tensor(drum_train, dtype=torch.float)
        bass_train = torch.tensor(bass_train, dtype=torch.float)
        drum_test = torch.tensor(drum_test, dtype=torch.float)
        bass_test = torch.tensor(bass_test, dtype=torch.float)
        
    examples_count = drum_train.size()[0]
    examples_id = 0
    
    running_loss = 0.0
    runnint_count = 0
    batch_id = 0
    while examples_id < examples_count:
        batch_drum_train = drum_train[examples_id:examples_id + batch_size,:,:].transpose(0,1)
        batch_bass_train = bass_train[examples_id:examples_id + batch_size,].transpose(0,1)
        # transpose нужен для обмена размерности батча и размерности шагов
#         print(f"batch_drum_train:{batch_drum_train.size()}, batch_bass_train:{batch_bass_train.size()}")

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        bass_outputs = dnb_gru(batch_drum_train).squeeze()
#         bass_outputs = bass_outputs.reshape(bass_outputs.size()[0], -1)
#         batch_bass_train = batch_bass_train.reshape(batch_bass_train.size()[0], -1)
#         print(f"bass_outputs:{bass_outputs.size()} batch_bass_train: {batch_bass_train.size()}")
#         print(f"bass_outputs:{bass_outputs} batch_bass_train: {batch_bass_train}")
        
        # loss = criterion(bass_outputs, batch_bass_train.long())
        loss = criterion(bass_outputs, batch_bass_train)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        runnint_count += 1
        period = 5
        if batch_id % period == 0 or examples_id + batch_size >= examples_count:
            print('[%d, %5d] train loss: %.7f' %
                  (epoch + 1, batch_id + 1, running_loss / runnint_count))
            running_loss = 0.0
            runnint_count = 1
            
        # update batch info
        examples_id += batch_size
        batch_id += 1
        
    # here we can insert measure error on test set

#should check accuracy on validation set
print('Finished Training')

shuffle_every_epoch is on
Epoch #0
[1,     1] train loss: 168.5638733
[1,     6] train loss: 127.1132634
[1,    11] train loss: 101.4188716
[1,    16] train loss: 92.2979444
[1,    21] train loss: 71.6840566
[1,    26] train loss: 71.7772776
[1,    31] train loss: 67.8496424
[1,    36] train loss: 62.4713039
[1,    41] train loss: 53.1411381
[1,    46] train loss: 54.5914408
[1,    51] train loss: 55.8497251
[1,    56] train loss: 52.2756551
[1,    61] train loss: 56.6614424
[1,    66] train loss: 49.4436226
[1,    71] train loss: 55.7260723
[1,    76] train loss: 45.9068794
[1,    81] train loss: 53.6488190
[1,    86] train loss: 53.5256379
[1,    91] train loss: 50.2961597
[1,    96] train loss: 42.8001207
[1,   101] train loss: 41.6890640
[1,   106] train loss: 58.1738612
[1,   111] train loss: 50.8313313
[1,   116] train loss: 46.1922646
[1,   121] train loss: 52.9857845
[1,   126] train loss: 44.0465450
[1,   131] train loss: 42.9023240
[1,   136] train loss: 47.5815996
[1,   141]

[8,   106] train loss: 52.6692117
[8,   111] train loss: 48.7638054
[8,   116] train loss: 43.4371694
[8,   121] train loss: 52.5173804
[8,   126] train loss: 37.8901126
[8,   131] train loss: 40.1063528
[8,   136] train loss: 40.6526419
[8,   141] train loss: 44.2088273
[8,   146] train loss: 50.7108065
[8,   150] train loss: 49.4578819
Epoch #8
[9,     1] train loss: 41.1341629
[9,     6] train loss: 43.4163367
[9,    11] train loss: 41.2518616
[9,    16] train loss: 39.8513254
[9,    21] train loss: 40.7009061
[9,    26] train loss: 45.2932657
[9,    31] train loss: 46.6525574
[9,    36] train loss: 43.6890831
[9,    41] train loss: 42.1405659
[9,    46] train loss: 50.2783813
[9,    51] train loss: 48.2332948
[9,    56] train loss: 43.3110034
[9,    61] train loss: 48.8320325
[9,    66] train loss: 44.9844227
[9,    71] train loss: 46.5210292
[9,    76] train loss: 36.9104258
[9,    81] train loss: 44.9857203
[9,    86] train loss: 44.8374926
[9,    91] train loss: 46.6656195
[9,  

[23,   106] train loss: 51.2264290
[23,   111] train loss: 46.4646072
[23,   116] train loss: 42.8978189
[23,   121] train loss: 51.8405024
[23,   126] train loss: 35.3677591
[23,   131] train loss: 38.0064443
[23,   136] train loss: 39.2328014
[23,   141] train loss: 41.8497645
[23,   146] train loss: 48.2690935
[23,   150] train loss: 50.5425629
Epoch #23
[24,     1] train loss: 35.4852676
[24,     6] train loss: 43.1181952
[24,    11] train loss: 42.2005151
[24,    16] train loss: 39.0810483
[24,    21] train loss: 38.6086826
[24,    26] train loss: 43.1419601
[24,    31] train loss: 47.5860589
[24,    36] train loss: 45.0622311
[24,    41] train loss: 39.6489347
[24,    46] train loss: 48.4742444
[24,    51] train loss: 46.8980319
[24,    56] train loss: 44.7761580
[24,    61] train loss: 48.1600164
[24,    66] train loss: 42.3307317
[24,    71] train loss: 44.9482746
[24,    76] train loss: 35.2784243
[24,    81] train loss: 41.7341118
[24,    86] train loss: 40.7522119
[24,    91

[38,    96] train loss: 36.5464611
[38,   101] train loss: 42.9617545
[38,   106] train loss: 50.8736159
[38,   111] train loss: 44.6010329
[38,   116] train loss: 41.4568971
[38,   121] train loss: 50.9660225
[38,   126] train loss: 34.1654714
[38,   131] train loss: 39.3129972
[38,   136] train loss: 39.8412247
[38,   141] train loss: 41.7290306
[38,   146] train loss: 47.8474181
[38,   150] train loss: 50.2671761
Epoch #38
[39,     1] train loss: 36.6905899
[39,     6] train loss: 40.1473325
[39,    11] train loss: 40.7774684
[39,    16] train loss: 40.3445886
[39,    21] train loss: 36.5641155
[39,    26] train loss: 42.9654274
[39,    31] train loss: 40.9887740
[39,    36] train loss: 46.4860910
[39,    41] train loss: 41.1435254
[39,    46] train loss: 47.7815603
[39,    51] train loss: 49.4640903
[39,    56] train loss: 44.6908258
[39,    61] train loss: 48.7498474
[39,    66] train loss: 42.0077934
[39,    71] train loss: 42.1444956
[39,    76] train loss: 34.6411648
[39,    81

[53,    86] train loss: 40.2620201
[53,    91] train loss: 45.3875287
[53,    96] train loss: 37.9410807
[53,   101] train loss: 40.5827249
[53,   106] train loss: 48.2353268
[53,   111] train loss: 47.1033669
[53,   116] train loss: 42.4475505
[53,   121] train loss: 50.0535749
[53,   126] train loss: 34.2914480
[53,   131] train loss: 38.2386621
[53,   136] train loss: 41.0038096
[53,   141] train loss: 39.3134149
[53,   146] train loss: 47.6220163
[53,   150] train loss: 49.3088577
Epoch #53
[54,     1] train loss: 38.2416267
[54,     6] train loss: 42.3386854
[54,    11] train loss: 41.6874104
[54,    16] train loss: 39.6638597
[54,    21] train loss: 39.0793107
[54,    26] train loss: 45.3977604
[54,    31] train loss: 44.6543433
[54,    36] train loss: 43.6545010
[54,    41] train loss: 40.4382394
[54,    46] train loss: 46.2518304
[54,    51] train loss: 46.9591643
[54,    56] train loss: 43.2355982
[54,    61] train loss: 49.2183901
[54,    66] train loss: 42.4130395
[54,    71

...

[432,    41] train loss: 36.9317525
[432,    46] train loss: 42.2093894
[432,    51] train loss: 40.8376236
[432,    56] train loss: 38.4401868
[432,    61] train loss: 44.1614984
[432,    66] train loss: 41.2225297
[432,    71] train loss: 40.3474871
[432,    76] train loss: 30.8870478
[432,    81] train loss: 35.7429682
[432,    86] train loss: 35.5225054
[432,    91] train loss: 41.5269121
[432,    96] train loss: 35.1441085
[432,   101] train loss: 37.0282024
[432,   106] train loss: 43.9918067
[432,   111] train loss: 40.3673623
[432,   116] train loss: 39.3617338
[432,   121] train loss: 39.2542737
[432,   126] train loss: 29.6174800
[432,   131] train loss: 32.5503642
[432,   136] train loss: 35.0555541
[432,   141] train loss: 40.0952937
[432,   146] train loss: 34.0745672
[432,   150] train loss: 44.7086044
Epoch #432
[433,     1] train loss: 35.8857536
[433,     6] train loss: 39.4669355
[433,    11] train loss: 35.7987372
[433,    16] train loss: 36.5640678
[433,    21] trai

[446,   121] train loss: 39.3071219
[446,   126] train loss: 30.4106661
[446,   131] train loss: 31.6134243
[446,   136] train loss: 36.2109114
[446,   141] train loss: 38.7421246
[446,   146] train loss: 34.7322187
[446,   150] train loss: 43.5034813
Epoch #446
[447,     1] train loss: 33.8091011
[447,     6] train loss: 37.9426804
[447,    11] train loss: 37.5222982
[447,    16] train loss: 36.0729001
[447,    21] train loss: 34.4708856
[447,    26] train loss: 38.3555876
[447,    31] train loss: 44.3442173
[447,    36] train loss: 35.7357445
[447,    41] train loss: 34.9086933
[447,    46] train loss: 43.2924525
[447,    51] train loss: 39.3010368
[447,    56] train loss: 37.2055175
[447,    61] train loss: 43.3443845
[447,    66] train loss: 38.1876481
[447,    71] train loss: 42.4111964
[447,    76] train loss: 30.7692626
[447,    81] train loss: 37.3696378
[447,    86] train loss: 33.8858649
[447,    91] train loss: 42.2604605
[447,    96] train loss: 34.5818698
[447,   101] trai

[461,    46] train loss: 42.6957410
[461,    51] train loss: 39.1680107
[461,    56] train loss: 37.0510400
[461,    61] train loss: 45.4004529
[461,    66] train loss: 37.5486520
[461,    71] train loss: 42.6013737
[461,    76] train loss: 30.9865452
[461,    81] train loss: 37.5879002
[461,    86] train loss: 35.1998177
[461,    91] train loss: 41.1738911
[461,    96] train loss: 36.1589069
[461,   101] train loss: 36.1628990
[461,   106] train loss: 43.0035788
[461,   111] train loss: 42.1344096
[461,   116] train loss: 38.0996494
[461,   121] train loss: 40.0794789
[461,   126] train loss: 29.0783720
[461,   131] train loss: 31.1670135
[461,   136] train loss: 37.0899426
[461,   141] train loss: 38.4446672
[461,   146] train loss: 33.7176275
[461,   150] train loss: 42.6386009
Epoch #461
[462,     1] train loss: 30.8396358
[462,     6] train loss: 38.2540023
[462,    11] train loss: 35.9642849
[462,    16] train loss: 34.7790292
[462,    21] train loss: 34.3341389
[462,    26] trai

[475,   126] train loss: 29.6385120
[475,   131] train loss: 31.7447348
[475,   136] train loss: 37.0218391
[475,   141] train loss: 38.4175485
[475,   146] train loss: 33.3684514
[475,   150] train loss: 43.1613777
Epoch #475
[476,     1] train loss: 34.5736427
[476,     6] train loss: 38.2276834
[476,    11] train loss: 36.2864456
[476,    16] train loss: 34.3301385
[476,    21] train loss: 34.2724133
[476,    26] train loss: 41.3712451
[476,    31] train loss: 40.2997653
[476,    36] train loss: 37.6926848
[476,    41] train loss: 38.3649502
[476,    46] train loss: 41.3980916
[476,    51] train loss: 40.9777813
[476,    56] train loss: 38.2048721
[476,    61] train loss: 42.5690943
[476,    66] train loss: 39.7435093
[476,    71] train loss: 40.4467392
[476,    76] train loss: 31.1646662
[476,    81] train loss: 35.9543521
[476,    86] train loss: 34.5877628
[476,    91] train loss: 41.1175893
[476,    96] train loss: 33.8936183
[476,   101] train loss: 35.1313521
[476,   106] trai

[490,    51] train loss: 41.0023136
[490,    56] train loss: 38.7683372
[490,    61] train loss: 44.3377406
[490,    66] train loss: 38.0881958
[490,    71] train loss: 39.2288268
[490,    76] train loss: 30.8484097
[490,    81] train loss: 36.7132848
[490,    86] train loss: 36.1060537
[490,    91] train loss: 41.5760167
[490,    96] train loss: 33.1836898
[490,   101] train loss: 37.2258142
[490,   106] train loss: 43.6945286
[490,   111] train loss: 41.7899412
[490,   116] train loss: 38.7544918
[490,   121] train loss: 38.7323742
[490,   126] train loss: 31.0454890
[490,   131] train loss: 31.8731772
[490,   136] train loss: 36.9730644
[490,   141] train loss: 37.2105172
[490,   146] train loss: 33.8548969
[490,   150] train loss: 42.9801971
Epoch #490
[491,     1] train loss: 32.4494591
[491,     6] train loss: 39.5604127
[491,    11] train loss: 35.1621628
[491,    16] train loss: 34.1306527
[491,    21] train loss: 33.5720421
[491,    26] train loss: 40.4526971
[491,    31] trai

In [18]:
batch_drum_train = drum_train[:,:,:].transpose(0,1)
batch_bass_train = bass_train[:,:].transpose(0,1)
with torch.no_grad():
    bass_outputs = dnb_gru(batch_drum_train)

In [19]:
result = bass_outputs.squeeze().int()
result

tensor([[ 5,  5,  5,  ...,  8,  8,  8],
        [ 3,  3,  3,  ...,  7,  7,  4],
        [ 3,  3,  3,  ...,  7,  7,  8],
        ...,
        [ 8,  8,  8,  ...,  6,  6,  8],
        [ 6,  6,  6,  ...,  2,  2,  3],
        [ 6,  6,  6,  ...,  3,  3, 10]], dtype=torch.int32)

Попробуем сохранить результаты работы сети. На anaconda нет mido, поэтому сохраняем результаты работы просто в массивчик npy... Однако, как альтернатива, его можно поставить чере pip в conda:
https://github.com/mido/mido/issues/198

In [26]:
import mido
from decode_patterns.data_conversion import build_track, DrumMelodyPair, NumpyImage, Converter


converter = Converter((16,50))

batch_drum = torch.cat((drum_train, drum_test, torch.tensor(drum_validation))).transpose(0,1)
batch_bass = torch.cat((bass_train.int(), bass_test.int(), torch.tensor(bass_validation).int())).transpose(0,1)
with torch.no_grad():
    bass_outputs = dnb_gru(batch_drum)
    bass_outputs = bass_outputs.squeeze().int()
    
    for i in range(bass_outputs.size()[1]):
        bass_seq = bass_outputs[:,i]
#         bass_seq = batch_bass[:,i]
#         print(f"bass_seq:{bass_seq.size()}")
        bass_output = []
        for bass_note in bass_seq:
            bass_row = np.eye(1, 36, bass_note - 1)[0]
            bass_output.append(bass_row)
        bass_output = torch.tensor(bass_output).int().squeeze()
#         print(f"bass_output:{bass_output.size()}")
        
#         print(f"batch_drum:{batch_drum[:,i,:].size()}, bass_output:{bass_output.size()}")
            
        img_dnb = torch.cat((batch_drum[:,i,:].int(),bass_output), axis=1)
#         print(f"img_dnb:{list(bass_output)}")
        numpy_pair = NumpyImage(np.array(img_dnb), 120, 1, 1, 36)
        pair = converter.convert_numpy_image_to_pair(numpy_pair)
#         print(f"pair.melody:{pair.melody}")
        mid = build_track(pair, tempo=pair.tempo)
        mid.save(f"midi/npy/sample{i+1}.mid")
#         np.save(f"midi/npy/drum{i+1}.npy", batch_drum[:,i,:].int())
#         np.save(f"midi/npy/bass{i+1}.npy", bass_outputs[:,i,:])